# Грузим библиотеки

In [ ]:
from flask import Flask
from flask import request
import requests
from flask import make_response
import os
import json
from pandas import DataFrame
import traceback


from methods import access
from methods.emoji import emoji
from methods import telegram_bot_methods
from methods import google_maps_methods
from methods import psql_methods
from methods import reply



import app



if __name__ == "__main__":

    
    app.application.run()



In [ ]:
X = {'update_id': 201011794, 'message': {'message_id': 159, 'from': {'id': 84723474, 'first_name': 'Александр', 'last_name': 'Толмачёв', 'language_code': 'ru-RU'}, 'chat': {'id': 84723474, 'first_name': 'Александр', 'last_name': 'Толмачёв', 'type': 'private'}, 'date': 1502611993, 'text': '/help', 'entities': [{'type': 'bot_command', 'offset': 0, 'length': 5}]}}
X['message']['text'] = '/start'
X['message']['chat']['last_name']

In [52]:
#тест отправки графика
import psycopg2
from pandas import DataFrame
import pandas as pd
from datetime import datetime
from datetime import timedelta

import numpy as np

#текущая метка времени
def now_str():
    now = datetime.now()
    now_str = str(now.year)+str(now.month if now.month >= 10 else  '0'+str(now.month))+str(now.day if now.day >= 10 else  '0'+str(now.day)) +' '+str(now.hour if now.hour >= 10 else  '0'+str(now.hour)) + str(now.minute if now.minute >= 10 else  '0'+str(now.minute)) + str(now.second if now.second >= 10 else  '0'+str(now.second))
    return now_str


dbname = 'dbr3jigs1op5oo'
port = '5432'
user = 'muwrkppfuyldmk'
host = 'ec2-54-227-252-202.compute-1.amazonaws.com'
password = '4c4eabfcaf92f7289ccfc1a314d04a3c3806db72b1bf12fd5f0f40c410b14355'

PSQL_heroku_keys = {'dbname' : dbname
                    , 'port' : port
                    , 'user' : user
                    , 'host' : host
                    , 'password' : password
                    }


user_id = 13
command = 'На текущий месяц'

conn = psycopg2.connect("dbname='%(dbname)s' port='%(port)s' user='%(user)s' host='%(host)s' password='%(password)s'" % PSQL_heroku_keys)



#формат ответа
response = {'status' : 200
            ,'report' : 'delete_transaction_plan'
            ,'system_message' : 'No report'
            ,'text' : None
            ,'reply_markup' : None
            }

# создаем запрос
cur = conn.cursor()

if command == 'На сегодня':
    cur.execute("select * from public.tasks where  user_id = %(user_id)s and date_task >= date_trunc('day', now()) and date_task < date_trunc('day', now()) +  interval '1 day'" % {  'user_id' : user_id} )
elif command == 'На текущую неделю':
    cur.execute("select * from public.tasks where  user_id = %(user_id)s and date_task >= date_trunc('week', now()) and date_task < date_trunc('week', now()) + interval '7 day'" % {  'user_id' : user_id} )
elif command == 'На текущий месяц':
    cur.execute("select * from public.tasks where  user_id = %(user_id)s and date_task >= date_trunc('month', now()) and date_task < date_trunc('month', now()) + interval '1 month'" % {  'user_id' : user_id} )



df_plan_tasks = DataFrame(cur.fetchall(), columns=[desc[0] for desc in cur.description]).sort_values(by='date_task' , ascending=True)

cur.close() 

if df_plan_tasks.shape[0] == 0:
    response['text'] = command + ' дел нет.'
else:
    text = command + ' следующие дела:\n\n'
    num = 1
    for row in df_plan_tasks.iterrows():
        text += str(num) + ': ' + row[1]['task'] + ' ' + row[1]['date_task'].strftime('%d.%m.%Y') + '\n'
        num += 1
        
        
    response['text'] = text
    


print ('ok')


df_plan_tasks

ok


,id,user_id,family_id,task,date_task,flg_done
1,2,13,None,Позвонить папе по поводу лоджии,2017-09-24,False
0,4,13,None,тестовые дела,2017-09-26,False


In [48]:
text = command + ' следующие дела:\n\n'
num = 1
for row in df_plan_tasks.iterrows():
    text += str(num) + ': ' + row[1]['task'] + ' ' + row[1]['date_task'].strftime('%d.%m.%Y') + '\n'

    num += 1


In [51]:
print (text)

На текущий месяц следующие дела:

1: Позвонить папе по поводу лоджии 24.09.2017
2: тестовые дела 26.09.2017



In [41]:
df_plan_tasks['date_task'].dt.strftime('%d.%m.%Y')

1    24.09.2017
0    26.09.2017
Name: date_task, dtype: object